In [4]:
import os
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data
from PIL import Image
import pickle
from tqdm import tqdm

# Load train data

In [5]:
train_dir = 'data/tiny-imagenet-200/train/'
train_dataset = datasets.ImageFolder(
    train_dir) # 
train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8)
toTensor = transforms.ToTensor()
toPIL = transforms.ToPILImage()
# print("加载训练目录: %s" % (train_dir))
# print("加载类别数量 %d"%len(train_dataset.classes))
# print("类别 n01443537 映射 index %d" % train_dataset.class_to_idx['n01443537'])
# print("图片 %s 映射到 idx %d" % train_dataset.imgs[0])

## Process test data

In [6]:
val_imgs = []
val_labels = []

val_dir = 'data/tiny-imagenet-200/val/'
with open(os.path.join(val_dir, 'val_annotations.txt'), "r") as f:
    val_f_annot = [(s.split('\t')[0], s.split('\t')[1]) for s in f.readlines()]
    
for img_file, class_name in val_f_annot:
    img = np.array(Image.open(os.path.join(val_dir, 'images', img_file)))
    if len(img.shape) != 3:
        img = img.reshape(64, 64, 1).repeat(3, axis=2)
    val_imgs.append(img)
    val_labels.append(train_dataset.class_to_idx[class_name])
# from utils import SubTrainDataset
# test_dataset = SubTrainDataset(val_imgs, val_labels, transform=transforms.ToTensor())
pickle.dump((val_imgs, val_labels), open("data/TinyImagenet_test.pkl", "wb"))

## Compute mean and std

In [8]:
X_tensor = []
for x, y in tqdm(train_dataset):
    X_tensor.append(toTensor(x))
X_tensor = torch.stack(X_tensor)

100%|██████████| 100000/100000 [00:18<00:00, 5501.44it/s]


In [9]:
# mean
X_tensor.mean(dim=[0, 2, 3]).numpy()

array([0.48024866, 0.44807237, 0.39754647], dtype=float32)

In [10]:
# mean
X_tensor.std(dim=[0, 2, 3]).numpy()

array([0.27698642, 0.2690645 , 0.2820819 ], dtype=float32)

# Random split train dataset

In [11]:
np.random.seed(0)
vic_idx = np.random.choice(len(train_dataset), len(train_dataset) // 2, replace=False)

In [12]:
X_list = []
y_list = []
# recover from np.array
# from PIL import Image
# im = Image.fromarray(X_train_list[0])
for x, y in tqdm(train_dataset):
    X_list.append(np.array(x))
    y_list.append(y)

100%|██████████| 100000/100000 [00:12<00:00, 7852.92it/s]


In [13]:
X_np = np.array(X_list)
y_np = np.array(y_list)

In [14]:
X_set1 = X_np[vic_idx]
y_set1 = y_np[vic_idx]
X_set2 = X_np[~vic_idx]
y_set2 = y_np[~vic_idx]

In [15]:
pickle.dump(((X_set1, y_set1), (X_set2, y_set2)), open("data/TinyImagenet_sub_train_split.pkl", "wb"))

## subset 1 mean and std

In [16]:
X_tensor[torch.tensor(vic_idx)].mean(dim=[0,2,3]).numpy()

array([0.4802226 , 0.44817278, 0.39785585], dtype=float32)

In [17]:
X_tensor[torch.tensor(vic_idx)].std(dim=[0,2,3]).numpy()

array([0.2769446 , 0.26894435, 0.28200737], dtype=float32)

## subset 2 mean and std

In [18]:
X_tensor[~torch.tensor(vic_idx)].mean(dim=[0,2,3]).numpy()

array([0.4803542 , 0.4477138 , 0.39684433], dtype=float32)

In [19]:
X_tensor[~torch.tensor(vic_idx)].std(dim=[0,2,3]).numpy()

array([0.27689543, 0.26896143, 0.28190473], dtype=float32)